In [70]:
# loading in modules
import sqlite3
# from peewee import *

import pandas as pd 
from sudulunu.helpers import pp, make_num, dumper, rc
import sys 

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

import os 
import pathlib
pathos = pathlib.Path(os.path.abspath('sent_sim.ipynb')).parent
os.chdir(pathos)
print(os.getcwd())


/Users/josh_nicholas/Repos/oz-2023-facebook-political-ads-scraper/classify_model


In [71]:
# from sklearn.datasets import fetch_20newsgroups
# categories = ['soc.religion.christian',
#               'comp.graphics']

# twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

# print(type(twenty_train))

from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples =1000, centers=2, random_state=42)

print(X)
print(y)

[[-1.67350551  7.88457927]
 [ 6.31757965  1.41958144]
 [ 3.54360375  0.5331188 ]
 ...
 [ 3.26797769  0.35960829]
 [ 5.76592909  1.92177596]
 [-0.31939469  8.20598784]]
[0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 1 0 1 0 1 0 1
 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 0 0 0
 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 0
 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0
 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1
 1 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1
 1 0 0 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 1 0 0
 1 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 1 0
 1 0 0 

In [72]:

db = 'input/scraperwiki.sqlite'
con = sqlite3.connect(db)

cur = con.cursor()

# table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# print(table_list)
# [('ads_by_id',), ('ads_by_query',)]

# key_list = [x for x in cur.execute('SELECT * FROM ads_by_query')]
entries = [x for x in cur.execute('SELECT * FROM ads_by_id')]

# advertisers = []
# for thingo in entries:
#     advertisers.append(thingo[2])
# print(list(set(advertisers)))

# ['From the Heart', 'Fair Australia', 'The Uluru Statement from the Heart', 'Senator Jacinta Nampijinpa Price', 
# 'Not My Voice', 'Referendum News', 'Linda Burney', 'Not Enough', 'Yes23']

con.close()

In [73]:
records = []

# dicto = {}
# X = []

# for thingo in entries[:2]:

#     ad_id = thingo[0]
#     advertiser_id = thingo[1]
#     name = thingo[2]
#     texto = thingo[3]
#     embeddings = model.encode([texto],convert_to_numpy=True)

#     dicto[str(embeddings[0])] = ad_id
#     X.append(embeddings)
#     # print(embeddings)

# # print(dicto)

# # sents = entries[:3]
# sents = entries.copy()
# sents = [x[3] for x in sents if x[3]]
# embeddings = model.encode(sents,convert_to_numpy=True)
# # print(embeddings)

lenno = int(len(entries)/4)
test = entries[:lenno]
train = entries[lenno:]

test = [x[3] for x in test if x[3]]
train = [x[3] for x in train if x[3]]

test = model.encode(test,convert_to_numpy=True)
train = model.encode(train,convert_to_numpy=True)


kmeans = KMeans(n_clusters=2, random_state=0, n_init='auto').fit(train)

In [77]:

traino = entries[0]

texto = traino[3]
embedo = model.encode([texto],convert_to_numpy=True)

# print(texto)
# print(kmeans.predict(embedo))

records = []

counter = 0
for thingo in entries:
    counter += 1
    if counter % 50 ==0:
        
        print(counter)
    old = pd.read_csv('inter/knn.csv')
    donners = old['Ad_id'].unique().tolist()
    ad_id = thingo[0]

    if ad_id in donners:
        continue
    else:
        try:
            ad_id = thingo[0]
            advertiser_id = thingo[1]
            name = thingo[2]
            texto = thingo[3]
            embedo = model.encode([texto],convert_to_numpy=True)
            prediction = kmeans.predict(embedo)

            records.append({"Ad_id": ad_id, "Advertiser": advertiser_id, "Prediction": prediction[0], "Text": texto})

            cat = pd.DataFrame.from_records(records)
            # new = pd.concat([old, cat])
            # new.drop_duplicates(subset=['Ad_id'], inplace=True)

            dumper('inter', 'knn', cat)

        except Exception as e:

            print(f"Exception is {e}")
            print(f"Line: {sys.exc_info()[-1].tb_lineno}")
            pp(cat)

pp(cat)





50
100
150
200
250
300
350
400
450
Exception is 'NoneType' object is not subscriptable
Line: 29
                Ad_id       Advertiser  Prediction  \
0    1083098386186304  102329728050606           1   
1    1356150461975366  102329728050606           1   
2     808801313988568  102329728050606           1   
3     839262740550948  102329728050606           1   
4    1024076745433089  102329728050606           1   
..                ...              ...         ...   
493   212097394873529  102329728050606           1   
494   218700897474857  102329728050606           1   
495   214119751244647  102329728050606           1   
496   244886638001228  102329728050606           1   
497  6419524001411199  102329728050606           1   

                                                  Text  
0    Listening to people leads to better outcomes o...  
1    A ‘Yes’ vote for constitutional recognition fo...  
2    A ‘Yes’ vote for constitutional recognition fo...  
3    A ‘Yes’ vote for const

In [78]:
keen = pd.read_csv('inter/knn.csv')

keen['Count'] = 1

grp = keen.groupby(by=['Prediction'])['Count'].sum().reset_index()

print(grp)

   Prediction  Count
0           0    116
1           1    849
